# Ejemplo de uso: ``pickle`` vs ``import``

En este notebook trataremos de diferenciar el uso de pickle de lo que podamos obtener importando una clase o una variable, cosa que parece haber creado confusión el último día.

Comencemos recordando qué hacíamos al importar una clase:
  - Cuando importas una clase que te has definido, estás cargando la estructura de un objeto. Al crearte una clase estás definiendo los atributos y métodos que tendrán todos los objetos que crees de esa clase, pero no guardarás ningún objeto en sí, solo la estructura.

  Al importar una variable con un valor determinado, sí que estaremos leyendo un valor, pero será una cosa aislada y que siempre tendrá el mismo valor, salvo que cambiemos el código a mano.
  
Por otra parte tendríamos pickle:
  - La librería ``pickle`` nos permite guardar un objeto de Python, ya sea una variable como la anterior o un objeto de una clase que nosotros queramos. La diferencia reside en que ahora lo que guardamos es el objeto con todos sus datos, hacemos una copia exacta de todo lo que tiene ese objeto, no de la estructura sino del objeto.
  
  Si bien es cierto que en el caso de la variable puede haber similitudes, pickle nos permite guardar el estado actual, por lo que el valor de la variable puede cambiar dinámicamente, sin falta de tener que cambiar el código manualmente. Es decir, me puedo definir una variable, hago unas operaciones con ella, cambio su valor, y me la guardo tal como está para leerla más adelante, ya sea otro día o en otro script, mediante la librería pickle. Luego puedo volver a modificarla y guardarla, de modo que es posible guardar el estado de la variable e ir cambiándolo según ejecute mi script, mientras que de la otra forma el valor será el mismo siempre que la lea, pues no puedo guardar un nuevo valor (sin cambiar a mano lo que está escrito en el código).
  
  Es muy útil para guardar estados, llevar variables que queremos que se guarden en diferentes sesiones o intercambiar información entre scripts.
  
  
  ``¡IMPORTANTE!`` La librería pickle permite guardar una imagen del objeto para poder leerla después pero NO guarda la clase, por lo que si queremos serializar (guardar con pickle) un objeto de una clase que nos hayamos creado, deberemos importarla con anterioridad a leer el objeto. 


A continuación, veremos un ejemplo muy snecillo donde probaremos la funcionalidad de pickle. Vamos a intentar hacer un pequeño juego que nos guarde los resultados:


## Piedra, papel o tijera (con memoria):

El enunciado de lo que querríamos hacer sería algo así:
  - Diseña un juego de piedra papel o tijera donde se vayan acumulando puntos en función de lo que seleccione el jugador.
  - Las reglas son las que ya conocemos:
    - Piedra > Tijera
    - Papel > Piedra
    - Tijera > Papel
  - Puntuaciones:
    - Si el jugador gana un duelo, suma 100 puntos.
    - Si el jugador saca lo mismo que la máquina: gana 25 puntos.
    - Si el jugador pierde, pierde 50 puntos.
  - La máquina seleccionará aleatoriamente su elección, mientras que el jugador lo hará mediante un menú que le pida que introduzca 1 (Piedra), 2 (Papel) o 3 (Tijera)
  - Si el jugador selecciona 0, termina el juego, que le pedirá un nombre para guardar su resultado en un diccionario
  - Si el jugador selecciona 4, podrá imprimir por pantalla los high scores (TOP 10), que deberán persisitir en memoria para que se mantengan otros días.
  

In [1]:
import numpy as np
import pandas as pd
import pickle
import datetime

In [2]:
# No lo hemos visto, pero para sacar la fecha utilizaremos esta expresión:
datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

'2020-10-27 19:39:59'

In [3]:
# Ejemplo de lectura de nuestro fichero:
with open('top_players', "rb") as f:
    df_top = pickle.load(f)
    
df_top

,nombre,score,partidas,fecha
0,Ejemplo,1175,47,2020-10-27 18:40:39
1,Señor Probabilidad,275,5,2020-10-27 00:12:13
2,Retro Gamer,250,5,1970-10-01 14:20:10
3,AAA,150,3,2020-10-27 00:09:37
4,AAA,125,2,2020-10-27 18:34:28
5,No funciona,100,4,2020-10-27 18:35:27
6,Playero,25,7,2020-10-27 00:01:58
7,AA,-25,2,2020-10-27 18:34:57


In [ ]:
posibilidades = [1, 2, 3]
posibilidades_txt = {1: 'piedra', 2: 'papel', 3: 'tijera'}

# # La primera vez, no nos quedará otra que crearnos el TOP (al que pondremos un valor por defecto, como solían hacer los arcades antiguos):
# df_top = pd.DataFrame([['Retro Gamer', 250, 5, '1970-10-01 14:20:10']], columns=['nombre', 'score', 'jugadas', 'fecha'])

# Pero en las posteriores iteraciones, lo leeremos mediante pickle para rescatar los resultados:
with open('top_players', "rb") as f:
    df_top = pickle.load(f)

print("Bienvenido al juego de Piedra, Papel o Tijera. El nuevo Among us!!")
msg_juego = "Para jugar, introduce un número del 0 al 4:\n0. Finalizar partida\n1. Piedra\n2. Papel\n3. Tijera\n4. Imprimir top\n"
print(msg_juego)

puntos = 0
partidas = 0

# Creamos el mensaje que vamos a reutilizar para jugar:
msg = "\nPiedra, papel o tijera. 1, 2, 3... ¡YA!"

while True:
    machine = np.random.choice(posibilidades)
    try:
        jugador = int(input(msg))
        if jugador < 0 or jugador > 4:
            print("ERROR. " + msg_juego)
        elif jugador == 0:
            if partidas == 0:
                print("Gracias por... ¿nada? Bye!")
                break
            # Sacamos posición (daremos prioridad a los más nevos):
            if len(df_top[df_top['score'] > puntos].index) == 0:
                # Si pasa esto, es el TOP 1:
                print("WOOOHOOOO!! Eres el campeón de esto!! Te pones en 1er lugar!!")
                nombre = input("Así que apunta orgulloso tu nombre: ")
            elif len(df_top[df_top['score'] > puntos].index) == len(df_top.index):
                print(f"A ver cómo te digo esto... No está mal, ¿vale? ¿Eres el último? Sí, pero, bueno, es que hay mucho nivel. Y quedar último no está tan mal.")
                nombre = input("Dame tu nombre antes de que me arrepienta, anda: ")
            else:
                pos = min(df_top[df_top['score'] > puntos].index) + 2 # 1 más porque empieza en el 0, y otro más porque el que estoy filtrando es mejor que yo
                print(f"¡Enhorabuena! Has quedado el {pos}º.")
                nombre = input("Graba tu nombre en el lugar que se merece: ")
            fecha = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            df_aux = pd.DataFrame([[nombre, puntos, partidas, fecha]], columns=['nombre', 'score', 'partidas', 'fecha'])
            df_top = df_top.append(df_aux).sort_values(by=['score', 'fecha'], ascending=[False, False]).reset_index(drop=True)
            break
        elif jugador == 4:
            print("TOP 10 Players:")
            print(df_top.iloc[:10])
        else:
            partidas += 1
            # Selección de la máquina:
            machine = np.random.choice(posibilidades)
            if jugador == machine:
                puntos += 25
                print(f"{posibilidades_txt[jugador]} == {posibilidades_txt[machine]}")
                print("EMPATE: 25 puntos para ti")
            elif (jugador == 1 and machine == 3) or (jugador == 2 and machine == 1) or (jugador == 3 and machine == 2):
                puntos += 100
                print(f"{posibilidades_txt[jugador]} > {posibilidades_txt[machine]}")
                print("VICTORIA: 100 puntos para ti")
            else:
                puntos -= 50
                print(f"{posibilidades_txt[jugador]} > {posibilidades_txt[machine]}")
                print("DERROTA: 50 puntos menos")
            print(f"Saldo:\n{puntos} puntos en {partidas} partidas")
    except:
        print("ERROR. ESO NO ES UN... ¡¡¡NÚMEROOOOOOO!!!!" + msg_juego)
        print()
        print(jugador)
        print(type(jugador))
            


# Al terminar, guardamos nuetro TOP:
with open('top_players', "wb") as f:
    pickle.dump(df_top, f)
        

In [7]:
with open('top_players', "rb") as f:
    df_top = pickle.load(f)
df_top

,nombre,score,partidas,fecha
0,Señor Probabilidad,275,5,2020-10-27 00:12:13
1,Retro Gamer,250,5,1970-10-01 14:20:10
2,AAA,150,3,2020-10-27 00:09:37
3,AAA,125,2,2020-10-27 18:34:28
4,No funciona,100,4,2020-10-27 18:35:27
5,Playero,25,7,2020-10-27 00:01:58
6,AA,-25,2,2020-10-27 18:34:57


Este es un ejemplo sencillo en el que estamos guardando un objeto y leyéndolo de nuevo otra vez. Sin embargo, en este caso estamos guardando simplemente un ``DataFrame``, pero podríamos guardar objetos mucho más complejos.

De hecho, un ``DataFrame`` podría ser guardado en un fichero (csv, txt...) para almacenar estos datos.


Si aún crees que se puede realizar este ejercicio con un import de variables, te insto a que lo hagas así ;).



## Ejercicio:

Replica el ejemplo anterior pero guardando (y leyéndolo) desde un fichero. Además, añade que se guarden las partidas ganadas, empatadas y perdidas:

Empieza creando un fichero csv con el dataframe y algunos registros:

In [6]:
# Esto no lo usaremos porque ya tenemos creado un ficherito :)

# df_top = pd.DataFrame([['Retro Gamer', 250, 5, 3, 2, 0, '1970-10-01 14:20:10']], columns=['nombre', 'score', 'partidas', 'ganadas', 'empatadas', 'perdidas', 'fecha'])
# # Escribimos a csv:
# df_top.to_csv("top.csv", sep = ";", index=False)

Continuamos con el script:

In [4]:
posibilidades = [1, 2, 3]
posibilidades_txt = {1: 'piedra', 2: 'papel', 3: 'tijera'}

# # La primera vez, no nos quedará otra que crearnos el TOP (al que pondremos un valor por defecto, como solían hacer los arcades antiguos):
# df_top = pd.DataFrame([['Retro Gamer', 250, 5, '1970-10-01 14:20:10']], columns=['nombre', 'score', 'jugadas', 'fecha'])

# Pero en las posteriores iteraciones, lo leeremos mediante pickle para rescatar los resultados:
df_top = pd.read_csv("top.csv", sep=';')

print("Bienvenido al juego de Piedra, Papel o Tijera. El nuevo Among us!!")
msg_juego = "Para jugar, introduce un número del 0 al 4:\n0. Finalizar partida\n1. Piedra\n2. Papel\n3. Tijera\n4. Imprimir top\n"
print(msg_juego)

puntos = 0
partidas = 0
ganadas = 0
empatadas = 0
perdidas = 0

# Creamos el mensaje que vamos a reutilizar para jugar:
msg = "\nPiedra, papel o tijera. 1, 2, 3... ¡YA!"

while True:
    try:
        jugador = int(input(msg))
        if jugador < 0 or jugador > 4:
            print("ERROR. " + msg_juego)
        elif jugador == 0:
            if partidas == 0:
                print("Gracias por... ¿nada? Bye!")
                break
            # Sacamos posición (daremos prioridad a los más nevos):
            if len(df_top[df_top['score'] > puntos].index) == 0:
                # Si pasa esto, es el TOP 1:
                print("WOOOHOOOO!! Eres el campeón de esto!! Te pones en 1er lugar!!")
                nombre = input("Así que apunta orgulloso tu nombre: ")
            elif len(df_top[df_top['score'] > puntos].index) == len(df_top.index):
                pos = len(df_top.index) + 1
                print(f"A ver cómo te digo esto... No está mal, ¿vale? ¿Eres el último? Sí, pero, bueno, es que hay mucho nivel. Y quedar {pos}º no está tan mal.")
                nombre = input("Dame tu nombre antes de que me arrepienta, anda: ")
            else:
                pos = len(df_top[df_top['score'] > puntos].index) + 2 # 1 más porque empieza en el 0, y otro más porque el que estoy filtrando es mejor que yo
                print(f"¡Enhorabuena! Has quedado el {pos}º.")
                nombre = input("Graba tu nombre en el lugar que se merece: ")
            fecha = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            df_aux = pd.DataFrame([[nombre, puntos, partidas, ganadas, empatadas, perdidas, fecha]], columns=['nombre', 'score', 'partidas', 'ganadas', 'empatadas', 'perdidas', 'fecha'])
            df_top = df_top.append(df_aux).sort_values(by=['score', 'fecha'], ascending=[False, False]).reset_index(drop=True)
            break
        elif jugador == 4:
            print("TOP 10 Players:")
            print(df_top.iloc[:10])
        else:
            partidas += 1
            # Selección de la máquina:
            machine = np.random.choice(posibilidades)
            if jugador == machine:
                empatadas += 1
                puntos += 25
                print(f"{posibilidades_txt[jugador]} == {posibilidades_txt[machine]}")
                print("EMPATE: 25 puntos para ti")
            elif (jugador == 1 and machine == 3) or (jugador == 2 and machine == 1) or (jugador == 3 and machine == 2):
                ganadas += 1
                puntos += 100
                print(f"{posibilidades_txt[jugador]} > {posibilidades_txt[machine]}")
                print("VICTORIA: 100 puntos para ti")
            else:
                perdidas += 1
                puntos -= 50
                print(f"{posibilidades_txt[jugador]} > {posibilidades_txt[machine]}")
                print("DERROTA: 50 puntos menos")
            print(f"Saldo:\n{puntos} puntos en {partidas} partidas")
    except:
        print("¡¡DAME UN NÚMERO, PLEASE!!")


# Al terminar, guardamos nuetro TOP:
df_top.to_csv("top.csv", sep = ";", index=False)
        

Bienvenido al juego de Piedra, Papel o Tijera. El nuevo Among us!!
Para jugar, introduce un número del 0 al 4:
0. Finalizar partida
1. Piedra
2. Papel
3. Tijera
4. Imprimir top




Piedra, papel o tijera. 1, 2, 3... ¡YA! 1


piedra > papel
DERROTA: 50 puntos menos
Saldo:
-50 puntos en 1 partidas



Piedra, papel o tijera. 1, 2, 3... ¡YA! 1


piedra == piedra
EMPATE: 25 puntos para ti
Saldo:
-25 puntos en 2 partidas


...


Piedra, papel o tijera. 1, 2, 3... ¡YA! aosknfl


¡¡DAME UN NÚMERO, PLEASE!!



Piedra, papel o tijera. 1, 2, 3... ¡YA! 0


¡Enhorabuena! Has quedado el 4º.


Graba tu nombre en el lugar que se merece:  AAA


In [6]:
print(df_top)

        nombre  score  partidas  ganadas  empatadas  perdidas  \
0           SS    750        24      8.0       10.0       6.0   
1  Retro Gamer    250         5      3.0        2.0       0.0   
2          AAA    175         4      2.0        1.0       1.0   
3          AAA     75         3      NaN        NaN       NaN   
4            a     25         1      0.0        1.0       0.0   

                 fecha  
0  2020-10-27 19:30:40  
1  1970-10-01 14:20:10  
2  2020-10-27 19:40:14  
3  2020-10-27 19:21:16  
4  2020-10-27 19:30:26  
